In [ ]:
import re
import numpy as np
import pickle
import os

In [ ]:
!pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 3.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Đường dẫn tới thư mục bạn muốn chuyển đến
path = '/content/drive/MyDrive/DACN3_Spell_VN/'

# Thực hiện việc chuyển đến thư mục
os.chdir(path)

Mounted at /content/drive


In [ ]:
!ls model

model_spell_ngram_4.h5	spell_0.9897.h5  spell_0.99.h5
model_spell_ngram_5.h5	spell_0.9899.h5  spell_ngram_5.h5


In [ ]:
from collections import Counter
from keras.models import load_model
from nltk.tokenize import word_tokenize
from nltk import ngrams, word_tokenize
import numpy as np
import re
import unidecode
import string
from tqdm import tqdm

import sys

if not sys.warnoptions:
    import warnings

    warnings.simplefilter("ignore")

model = load_model("model/model_spell_ngram_5.h5")

NGRAM = 5
MAXLEN = 40
alphabet = ['\x00', ' ', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r',
            's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
            'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7',
            '8', '9', 'á', 'à', 'ả', 'ã', 'ạ', 'â', 'ấ', 'ầ', 'ẩ', 'ẫ', 'ậ', 'ă', 'ắ', 'ằ', 'ẳ', 'ẵ', 'ặ', 'ó', 'ò',
            'ỏ', 'õ', 'ọ', 'ô', 'ố', 'ồ', 'ổ', 'ỗ', 'ộ', 'ơ', 'ớ', 'ờ', 'ở', 'ỡ', 'ợ', 'é', 'è', 'ẻ', 'ẽ', 'ẹ', 'ê',
            'ế', 'ề', 'ể', 'ễ', 'ệ', 'ú', 'ù', 'ủ', 'ũ', 'ụ', 'ư', 'ứ', 'ừ', 'ử', 'ữ', 'ự', 'í', 'ì', 'ỉ', 'ĩ', 'ị',
            'ý', 'ỳ', 'ỷ', 'ỹ', 'ỵ', 'đ', 'Á', 'À', 'Ả', 'Ã', 'Ạ', 'Â', 'Ấ', 'Ầ', 'Ẩ', 'Ẫ', 'Ậ', 'Ă', 'Ắ', 'Ằ', 'Ẳ',
            'Ẵ', 'Ặ', 'Ó', 'Ò', 'Ỏ', 'Õ', 'Ọ', 'Ô', 'Ố', 'Ồ', 'Ổ', 'Ỗ', 'Ộ', 'Ơ', 'Ớ', 'Ờ', 'Ở', 'Ỡ', 'Ợ', 'É', 'È',
            'Ẻ', 'Ẽ', 'Ẹ', 'Ê', 'Ế', 'Ề', 'Ể', 'Ễ', 'Ệ', 'Ú', 'Ù', 'Ủ', 'Ũ', 'Ụ', 'Ư', 'Ứ', 'Ừ', 'Ử', 'Ữ', 'Ự', 'Í',
            'Ì', 'Ỉ', 'Ĩ', 'Ị', 'Ý', 'Ỳ', 'Ỷ', 'Ỹ', 'Ỵ', 'Đ']
letters = list(
    "abcdefghijklmnopqrstuvwxyzáàảãạâấầẩẫậăắằẳẵặóòỏõọôốồổỗộơớờởỡợéèẻẽẹêếềểễệúùủũụưứừửữựíìỉĩịýỳỷỹỵđABCDEFGHIJKLMNOPQRSTUVWXYZÁÀẢÃẠÂẤẦẨẪẬĂẮẰẲẴẶÓÒỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢÉÈẺẼẸÊẾỀỂỄỆÚÙỦŨỤƯỨỪỬỮỰÍÌỈĨỊÝỲỶỸỴĐ")
accepted_char = list((string.digits + ''.join(letters)))


def extract_phrases(text):
    pattern = r'\w[\w ]*\w|\s\W+|\W+|\n'
    return re.findall(pattern, text)


def encoder_data(text, maxlen=MAXLEN):
    text = "\x00" + text
    x = np.zeros((maxlen, len(alphabet)))
    for i, c in enumerate(text[:maxlen]):
        x[i, alphabet.index(c)] = 1
    if i < maxlen - 1:
        for j in range(i + 1, maxlen):
            x[j, 0] = 1
    return x


def decoder_data(x):
    x = x.argmax(axis=-1)
    return ''.join(alphabet[i] for i in x)


def nltk_ngrams(words, n=5):
    return ngrams(words.split(), n)


def guess(ngram):
    # Example: ngram = ('Tôi', 'ddi', 'học', 'ở', 'đâu')
    text = ' '.join(ngram)
    preds = model.predict(np.array([encoder_data(text)]), verbose=0)
    return decoder_data(preds[0]).strip('\x00')


def correct(sentence):
    # Lặp qua từng ký tự trong câu.
    for i in sentence:
        # Kiểm tra xem ký tự hiện tại có trong danh sách các ký tự được chấp nhận hay không.
        # Nếu không, thay thế ký tự đó bằng khoảng trắng.
        if i not in accepted_char:
            sentence = sentence.replace(i, " ")
    # Kiểm tra số từ trong câu
    num_words = len(sentence.split())

    # Nếu số từ không đủ để tạo n-gram, trả về câu không đổi
    if num_words < NGRAM:
      ngrams = list(nltk_ngrams(sentence, n=num_words))
      # Đoán các từ trong các n-gram.
      guessed_ngrams = list(guess(ngram) for ngram in ngrams)
      # Tạo một danh sách các bảng ứng viên từ.
      candidates = [Counter() for _ in range(len(guessed_ngrams) + num_words - 1)]

      # Lặp qua các n-gram đã đoán.
      for nid, ngram in (enumerate(guessed_ngrams)):
          # Tách các từ từ n-gram.
          for wid, word in (enumerate(re.split(' +', ngram))):
              # Cập nhật bảng ứng viên từ với từ hiện tại.
              candidates[nid + wid].update([word])
      # Xây dựng câu đúng bằng cách chọn từ phổ biến nhất từ mỗi bảng ứng viên từ.
      output = ' '.join(c.most_common(1)[0][0] for c in candidates)
    else:
      # Tạo các n-gram từ câu đã được xử lý.
      ngrams = list(nltk_ngrams(sentence, n=NGRAM))
      # Đoán các từ trong các n-gram.
      guessed_ngrams = list(guess(ngram) for ngram in ngrams)
      # Tạo một danh sách các bảng ứng viên từ.
      candidates = [Counter() for _ in range(len(guessed_ngrams) + NGRAM - 1)]

      # Lặp qua các n-gram đã đoán.
      for nid, ngram in (enumerate(guessed_ngrams)):
          # Tách các từ từ n-gram.
          for wid, word in (enumerate(re.split(' +', ngram))):
              # Cập nhật bảng ứng viên từ với từ hiện tại.
              candidates[nid + wid].update([word])
      # Xây dựng câu đúng bằng cách chọn từ phổ biến nhất từ mỗi bảng ứng viên từ.
      output = ' '.join(c.most_common(1)[0][0] for c in candidates)
    return output

def check_text(regex, text):
    pattern = re.compile(regex)
    if pattern.fullmatch(text):
        return True
    else:
        return False
def correct_2(text):
  ep = extract_phrases(text)
  rs_ = ""
  for value in ep:
    regex = r'\w[\w ]+'
    if check_text(regex, value):
      rs_ = rs_ + correct(value)
    else:
      rs_ = rs_ + value
  return rs_

In [ ]:
raw_text = "Bất cuws lúc nao, trong cuộc phieeu lưu đơn thương độc max"
results = correct(raw_text)
results

In [ ]:
text= "Bất cuws lúc nao, trong cuộc phieeu lưu đơn thương độc max. Tôi ddi học ở đaau. Cơn mưa ngang qua,  cơn muwa  @ ngang qua\nBất cuws lúc nao, trong cuộc phieeu lưu đơn thương độc max. Tôi ddi học ở đaau. Cơn mưa ngang qua,  cơn muwa  @ ngang qua"
results = correct_2(text)
results

In [ ]:
import re

# Chuỗi có nhiều dòng
chuoi = """Dòng đầu tiên
Dòng thứ hai
Dòng thứ ba"""

# Sử dụng re.split để tách chuỗi dựa trên ký tự xuống dòng
ket_qua = re.split("\n", chuoi)

print(ket_qua)  # Kết quả: ['Dòng đầu tiên', 'Dòng thứ hai', 'Dòng thứ ba']

In [ ]:
extract_phrases(text)